# Set up HLA typing

In [1]:
#setwd("/frazer01/projects/CEGS/analysis/hla_type_1kgp/")
setwd("/frazer01/home/aphodges/software/my_hla_typing/")
source("1kgp/functions.R")

In [5]:
dir()

[1] "1kgp"                "hla_typing.sh"       "HLAVBSeq.jar"       
 [4] "input"               "nohup.out"           "pipeline"           
 [7] "process_hla_types.R" "README.md"           "reference"          
[10] "TMP"

In [4]:
dir.create("pipeline/hla_typing"                , showWarnings = FALSE)
dir.create("pipeline/hla_typing/logs"           , showWarnings = FALSE)
dir.create("pipeline/hla_typing/processing"     , showWarnings = FALSE)
#dir.create("pipeline/hla_typing/hlavbseq_output", showWarnings = FALSE)


In [6]:
id2url_2504           = fread("input/1000G_2504_high_coverage.sequence.index"        , sep = "\t", header = TRUE , data.table = FALSE)[,c("#ENA_FILE_PATH", "RUN_ID", "LIBRARY_NAME", "POPULATION")]
id2url_698            = fread("input/1000G_698_related_high_coverage.sequence.index" , sep = "\t", header = FALSE, data.table = FALSE)[,c(1,3,10,11)]
colnames(id2url_2504) = c("cram_url", "run_id", "id", "population")
colnames(id2url_698 ) = c("cram_url", "run_id", "id", "population")
id2url                = rbind(id2url_2504, id2url_698)

fwrite(id2url, "pipeline/hla_typing/id2url.txt", sep = "\t", col.names = TRUE, row.names = FALSE)

ERROR: Error in fread("input/1000G_2504_high_coverage.sequence.index", sep = "\t", : File 'input/1000G_2504_high_coverage.sequence.index' does not exist or is non-readable. getwd()=='/frazer01/home/aphodges/software/my_hla_typing'


# Genome BED file for filtering:
- chr6:25000000-34000000
- chr6_GL000250v2_alt
- chr6_GL000251v2_alt
- chr6_GL000252v2_alt
- chr6_GL000253v2_alt
- chr6_GL000254v2_alt
- chr6_GL000255v2_alt
- chr6_GL000256v2_alt
- chr6_KI270758v1_alt
- chr6_GL000256v2_alt
- chr6_GL000256v2_alt
- chr6_GL000256v2_alt
- chr6_GL000256v2_alt
- EVerything HLA

# Run HLA typing

In [4]:
run_hla_typing_qsub = function(id2url, qsub = FALSE)
{
    sh_file = paste(getwd(), "pipeline/hla_typing/run_hla_typing.sh", sep = "/")
    
    writeLines(text = c("#!/usr/bin/sh",
                        "source /frazer01/home/matteo/.bashrc",
                        "export REF_PATH=/frazer01/projects/CEGS/analysis/hla_type_1kgp/input/cram_cache/cache/%2s/%2s/%s:http://www.ebi.ac.uk/ena/cram/md5/%s",
                        "export REF_CACHE=/frazer01/projects/CEGS/analysis/hla_type_1kgp/input/cram_cache/cache/%2s/%2s/%s",
                        paste("Rscript", paste(paste(getwd(), "script", "run_hla_typing.R", sep = "/"),
                                               "--taskid"   , "$SGE_TASK_ID")
                             )
                       ), 
               con  = sh_file, 
               sep  = "\n\n")
    
    qsub_command = paste("qsub",
                         "-t", paste(1, "-", nrow(id2url), ":1", sep = ""),
                         "-tc", 50, 
                         "-l" , "short",
                         "-l" , "h_vmem=8G",
                         "-pe", "smp", 2,
                         "-o" , paste(getwd(), "pipeline/hla_typing/logs.out", sep = "/"),
                         "-e" , paste(getwd(), "pipeline/hla_typing/logs.err", sep = "/"),
                         sh_file
                        )
    
    message(qsub_command)
    if(qsub == TRUE){system(qsub_command)}
}

run_hla_typing_qsub(id2url, FALSE)


qsub -t 1-3202:1 -tc 50 -l short -l h_vmem=8G -pe smp 2 -o /frazer01/projects/CEGS/analysis/hla_type_1kgp/pipeline/hla_typing/logs.out -e /frazer01/projects/CEGS/analysis/hla_type_1kgp/pipeline/hla_typing/logs.err /frazer01/projects/CEGS/analysis/hla_type_1kgp/pipeline/hla_typing/run_hla_typing.sh



# Progress

In [5]:
infiles           = list.files("pipeline/hla_typing/processing", pattern = "hla_types.txt", full.names = FALSE)


In [6]:
message(paste(paste ("HLA typed individuals", length(infiles), sep = " = "), 
              paste0("Processed = "         , signif(length(infiles) / nrow(id2url) * 100, digits = 3), "%"), 
              sep = "\n"), appendLF = FALSE)

HLA typed individuals = 3187
Processed = 99.5%


# Gene info

In [21]:
geneinfo = fread("/frazer01/reference/private/Gencode.v34lift37/gene_info.txt", sep = "\t", header = TRUE, data.table = FALSE)
genes    = fread("pipeline/hla_typing/processing/HG00096.hla_types.txt", sep = "\t", header = TRUE, data.table = FALSE)[,1]
genes    = ifelse(genes %in% c("HFE", "MICA", "MICB", "TAP1", "TAP2"), yes = genes, no = paste("HLA", genes, sep = "-"))
geneinfo = geneinfo[ geneinfo$gene_name %in% genes, c("gene_id", "gene_name", "chrom", "start", "end", "strand")]
geneinfo = rbind(geneinfo, data.frame(gene_id   = c("uc011fni.2", "uc011hzq.2", "100127173"),
                                      gene_name = c("HLA-DRB3", "HLA-DRB4", "HLA-Y"),
                                      chrom     = "chr6", 
                                      start     = c(28477797 + 3934127, 28696604 + 3882324, 29910341),
                                      end       = c(28477797 + 3947195, 28696604 + 3897292, 29913232),
                                      strand    = c("-", "-", "+")
                                     ))
geneinfo$gene = sub("HLA-", "", geneinfo$gene_name)

fwrite(geneinfo, "pipeline/hla_typing/gene_info.txt", sep = "\t", col.names = TRUE, row.names = FALSE)

In [18]:
setdiff(genes, geneinfo$gene_name)

[1] "HLA-DRB3" "HLA-DRB4" "HLA-Y"

# SCRATCH

In [20]:
run_hla_typing = function(taskid, ids)
{
    id      = ids[[taskid]]
    prefix  = paste("pipeline/hla_typing/processing", id, sep = "/")
    cram    = paste(cram_folder, paste(id, "cram", sep = "."), sep = "/")
    command = paste("bash", 
                    "/frazer01/home/matteo/software/my_hla_typing/hla_typing.sh",
                    cram,
                    "hg38_v02",
                    prefix,
                    "nuc_red",
                    "/frazer01/home/matteo/software/my_hla_typing/reference/nuc_red/alleles.nuc_red.txt",
                    5,
                    76
                   )
    
    message(command)
}


taskid = 1

run_hla_typing(taskid, ids)


bash /frazer01/home/matteo/software/my_hla_typing/hla_typing.sh /sdsc/cast/ukbb/exome/fe_crams/2053937.cram hg38_v02 pipeline/hla_typing/processing/2053937 nuc_red /frazer01/home/matteo/software/my_hla_typing/reference/nuc_red/alleles.nuc_red.txt 40 76



In [ ]:
bash /frazer01/home/matteo/software/my_hla_typing/hla_typing.sh CRAM hg38 /frazer01/projects/CEGS/analysis/hla_vbseq_pipeline/pipeline/hla_vbseq/out/VTE.070.nuc_red nuc_red /frazer01/home/matteo/software/my_hla_typing/reference/nuc_red/alleles.nuc_red.txt 25 76


In [22]:
ukbb_pgen_prefix

ERROR: Error in eval(expr, envir, enclos): object 'ukbb_pgen_prefix' not found
